In [1]:
import os, re, json
import base64
import fireworks.client

In [2]:
def load_config(config_file='config.json'):
    with open(config_file, 'r') as f:
        config = json.load(f)
    return config

In [3]:
config = load_config()
api_key = open(config['fireworks_api_key_file'], "r").read().strip("\n")
fireworks.client.api_key = api_key

In [4]:
def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

In [5]:
def extract_number_from_text(text):
    # Find all numbers in the text
    text = text[:500] 
    numbers = re.findall(r'\d+', text)
    if not numbers:
        return None
    # Return the first number found
    return int(numbers[0])

In [6]:
def get_vlm_result(base64_image, debug=False):
    det_prompt = "You are a satellite imagery analyst. Look at the satellite image and count the number of distinct buildings in the picture. Do not count landscape features or vehicles, only manmade buildings such as houses or other roofed structures. Please provide only the number, not an explanation or any other text."
    response = fireworks.client.ChatCompletion.create(
        model="accounts/fireworks/models/qwen2-vl-72b-instruct",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": det_prompt},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}"
                        },
                    },
                ],
            }
        ],
        stream=False,
    )
    if debug:
        print(response.choices[0].message.content)
    clean_response = extract_number_from_text(response.choices[0].message.content)
    return clean_response

In [7]:
run_num = '1'
image_dir = config['cropped_jpg_location']
folder_path = 'qwen72_outputs/'+run_num

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for filename in os.listdir(image_dir):
    base64_image = encode_image(image_dir+'/'+filename)
    base_name = filename.split('_pre_disaster.jpg')[0]
    response = get_vlm_result(base64_image, debug=False)
    if response:
        with open(folder_path+'/'+base_name+'.txt', 'w') as file:
            file.write(str(response))

In [8]:
run_num = '2'
image_dir = config['cropped_jpg_location']
folder_path = 'qwen72_outputs/'+run_num

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for filename in os.listdir(image_dir):
    base64_image = encode_image(image_dir+'/'+filename)
    base_name = filename.split('_post_disaster.jpg')[0]
    response = get_vlm_result(base64_image, debug=False)
    if response:
        with open(folder_path+'/'+base_name+'.txt', 'w') as file:
            file.write(str(response))